In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#this model is  inspired from 
#https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

In [0]:
#https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17

In [96]:

##NOT FOR SYSTEMS
!unzip -P ****** -qq '/content/drive/My Drive/EnglishData/task_c_distant.zip' 

replace task_c_distant_ann.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:

import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import numpy as np

In [0]:

def convertToFloat(val):
    if not val:
        return 0    
    try:
        return np.float64(val)
    except:        
        return np.float64(0)


headers=['id','text','average_ind','average_grp','average_oth','std_ind','std_grp','std_oth']
taskc = pd.read_csv("task_c_distant_ann.tsv", delimiter='\t',names=headers,low_memory=False,converters={"average_ind":convertToFloat,
                                                                                                    "average_grp":convertToFloat,
                                                                                                    "average_oth":convertToFloat,
                                                                                                    "std_ind":convertToFloat,
                                                                                                    "std_grp":convertToFloat,
                                                                                                    "std_oth":convertToFloat})
taskc=taskc[1:]

In [99]:
taskc.head()

id  ...   std_oth
1  1159533712079503361  ...  0.138649
2  1159533713044234241  ...  0.083230
3  1159533718345830400  ...  0.254621
4  1159533739871002625  ...  0.067552
5  1159533742366633984  ...  0.104436

[5 rows x 8 columns]

In [100]:
taskc.values

array([['1159533712079503361',
        '@USER Trump is a fucking idiot his dementia is getting worse',
        0.8334315660070198, ..., 0.20833409336419706,
        0.09893746284301566, 0.1386493230104102],
       ['1159533713044234241',
        '@USER HELL YES! His grinned and thumbs up are disgusting!',
        0.48106237178573397, ..., 0.3452248610539589,
        0.33592350092689993, 0.08323036242339713],
       ['1159533718345830400',
        "@USER Can't wait to see the shit show his death will bring.",
        0.4388127663192773, ..., 0.18260870386171185,
        0.18688048564468512, 0.25462109958562573],
       ...,
       ['1187636295306268673',
        'I meant tool but the point is women ain’t shit. 🙅🏻\u200d♂️🤷🏻\u200d♂️',
        0.36375786996300313, ..., 0.23943555651707127,
        0.16535075005267147, 0.09506541000311416],
       ['1187636299785744384', '@USER i sang too the fuck-',
        0.5982300263631064, ..., 0.19187065632830977,
        0.10707485138459394, 0.115205

In [0]:

dfnumpy=taskc.to_numpy();
x=dfnumpy[:, 1].reshape(-1, 1)
y_train1=dfnumpy[:, 2].reshape(-1, 1)
y_train2=dfnumpy[:, 3].reshape(-1, 1)
y_train3=dfnumpy[:, 4].reshape(-1, 1)



In [0]:
y=np.stack((y_train1.flatten(),y_train2.flatten(),y_train3.flatten()), axis=-1, out=None)

In [103]:
y_train2

array([[0.07610979866070268],
       [0.3673629474131143],
       [0.2685740368872388],
       ...,
       [0.43151670865663655],
       [0.16155983352929942],
       [0.15227155079903518]], dtype=object)

In [104]:
y[0:5]

array([[0.8334315660070198, 0.07610979866070268, 0.10776452321727449],
       [0.48106237178573397, 0.3673629474131143, 0.13884137991817902],
       [0.4388127663192773, 0.2685740368872388, 0.3775726696311655],
       [0.7129954339515537, 0.12350408671699416, 0.11113008071093726],
       [0.6914140593727042, 0.14672323837904971, 0.19228166422122278]],
      dtype=object)

In [105]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords 
import re
import string
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)
stopwords=set(stopwords.words('english'))
def preprocessinglib(arrt):
    ans =[]
    for txt in arrt:
        unique_words = dict.fromkeys(txt.split())
        txt2=' '.join(unique_words)
        tokens=tknzr.tokenize(txt2)
        ctokens=[]
        for t in tokens:
          if  t not in stopwords and t not in string.punctuation :
            ctokens.append(t)
        clean_s = ' '.join(ctokens)
        ans.append(clean_s)
               
    return ans


In [0]:
preprocessedTweets=preprocessinglib(x[:,0])

In [0]:
from sklearn.utils import shuffle
preprocessedTweets,y=shuffle(preprocessedTweets,y)

In [109]:
preprocessedTweets

['I fucking hate f gsThoughts LGBT community',
 'That shit 🔥 \U0001f975',
 'Kill fool thinking bout shit earlier 💯 💯 💯 We going talk fr tomorrow Need get started',
 'fuck lettuce Jake Think needs keep opinions future 🤣',
 'Using campaign .. sick bastard',
 'Bitch I ’ mf ass 😭 😭 😭 used think tire gone fly tf',
 'Trump respect fucking office',
 "I can't stop bitch today everyone pissing Can fucking already",
 "OK He's still fucking liberal probably curb immigration",
 'I type fast fuck oops',
 'I laugh lot shit 😂',
 '“ How feel bad bout wantin sum different dick cause I ’ gon fuck anotha nigga I ’ getting pussy ”',
 'Bitch 😂 😂 😂 fuck',
 'Cold fuck lol bring hella jackets',
 'My thoughts well Do ever wonder I full shit',
 'How hell truth racist',
 'looks like imma freezing ass weekend \U0001f976 \U0001f92a 😂',
 'uploaded one nudes story 😭 😭 deleted bitch QUICK 😭 😭 😭',
 'shit hurts I deserved 😭',
 'tfw enough shit',
 'Kids really sex middle school \U0001f974 thing I fucking ugly 🤣',
 'coul

In [0]:
# Import the dependencies
from keras.models import Sequential
from keras.layers import Dense, LSTM,SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [0]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [113]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(preprocessedTweets)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 60150 unique tokens.


In [114]:
X = tokenizer.texts_to_sequences(preprocessedTweets)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (188973, 250)


In [115]:
y.shape

(188973, 3)

In [0]:
x_train=X
y_train=y

In [0]:

#GET THE DATA FROM THE PANDAS FRAME
headers=['id','tweet','subtask_a','subtask_b','subtask_c']
englishdata = pd.read_csv("/content/drive/My Drive/EnglishData/OLIDv1.0/olid-training-v1.0.tsv", delimiter='\t',names=headers,low_memory=False)
englishdata=englishdata[['id','tweet','subtask_c']]
englishdata=englishdata[1:]

In [0]:
englishdata = englishdata.dropna(subset=['subtask_c'])

In [119]:
englishdata.head()

id                                              tweet subtask_c
2   90194  @USER @USER Go home you’re drunk!!! @USER #MAG...       IND
6   97670                  @USER Liberals are all Kookoo !!!       OTH
8   52415  @USER was literally just talking about this lo...       GRP
10  13384  @USER Canada doesn’t need another CUCK! We alr...       IND
13  28414  @USER you are a lying corrupt traitor!!! Nobod...       IND

In [0]:
import tensorflow as tf

In [0]:
dfnumpy=englishdata.to_numpy()
x_test=dfnumpy[:, 1].reshape(-1, 1)
y_test=dfnumpy[:, 2].reshape(-1, 1)

In [122]:
y_test

array([['IND'],
       ['OTH'],
       ['GRP'],
       ...,
       ['GRP'],
       ['IND'],
       ['OTH']], dtype=object)

In [123]:
y_test.flatten()

array(['IND', 'OTH', 'GRP', ..., 'GRP', 'IND', 'OTH'], dtype=object)

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def f1_calc(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=np.argmax(predictions,axis=1).to('cpu').numpy().flatten()
  labelsFlattend=np.argmax(predictions,axis=1).to('cpu').numpy().flatten()
  print(predictions)
  print("Predictions Argmax",predictionArgmax)
  print("labels Flattened",labelsFlattend)   
  return f1_score(labelsFlattend, predictionArgmax, average='macro')

In [125]:
len(y_test)

3876

In [0]:
#f1 inspired from 
#https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

In [127]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=x_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 3
batch_size = 64

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 170075 samples, validate on 18898 samples
Epoch 1/3
170075/170075 [==============================] - 1663s 10ms/step - loss: 1.0369 - acc: 0.8641 - val_loss: 1.0305 - val_acc: 0.8744
Epoch 2/3
170075/170075 [==============================] - 1660s 10ms/step - loss: 1.0281 - acc: 0.8793 - val_loss: 1.0299 - val_acc: 0.8764
Epoch 3/3
170075/170075 [==============================] - 1668s 10ms/step - loss: 1.0255 - acc: 0.8865 - val_loss: 1.0302 - val_acc: 0.8778


In [128]:
x_test

array([['@USER @USER Go home you’re drunk!!! @USER #MAGA #Trump2020 👊🇺🇸👊 URL'],
       ['@USER Liberals are all Kookoo !!!'],
       ['@USER was literally just talking about this lol all mass shootings like that have been set ups. it’s propaganda used to divide us on major issues like gun control and terrorism'],
       ...,
       ['@USER @USER @USER @USER Liars like the Antifa twins you vigorously defend?'],
       ['@USER Sometimes I get strong vibes from people and this man’s vibe is tens of millions of murders - he is more dangerous than DT.'],
       ["@USER And why report this garbage.  We don't give a crap."]],
      dtype=object)

In [0]:
preprocessedTweets1=preprocessinglib(x_test[:,0])

In [130]:
X_1 = tokenizer.texts_to_sequences(preprocessedTweets1)
X_test = pad_sequences(X_1, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Shape of data tensor: (3876, 250)


In [0]:
def giveLabel2(y1):
  i=0
  largest_index=0
  #label1=['IND','GRP','OTH']
  ylabels=[]
  for r in y1:
    if (r=="IND"): 
        largest_index = 0 
    elif (r=="GRP"):
        largest_index = 1 
    else: 
        largest_index = 2 
    
    ylabels.append(largest_index)
    i=i+1;
  return ylabels

In [0]:
yz=giveLabel2(y_test)

In [133]:
set(yz)

{0, 1, 2}

In [0]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(yz)
encoded_Y = encoder.transform(yz)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [135]:
dummy_y

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

In [136]:
pip install h5py

In [137]:
encoder.classes_

array([0, 1, 2])

In [0]:

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("SubtaskCEnglishData.h5")

In [0]:
from keras.models import model_from_json


In [140]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("SubtaskCEnglishData.h5")
print("Loaded model from disk")

Loaded model from disk


In [141]:
score= model.evaluate(X_test,dummy_y)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(score[0],score[1]))

3876/3876 [==============================] - 9s 2ms/step
Test set
  Loss: 0.748
  Accuracy: 0.755


In [0]:
yvalid=model.predict_classes(X_test)

In [143]:
type(X_test)

numpy.ndarray

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

def calculateF1Score(predictions,labels):
  #rowwise return the index of the max element ie 0 or 1 depending on the maximum value returned
  predictionArgmax=predictions.flatten()
  labelsFlattend=labels.flatten()
  #print("Predictions Argmax",predictionArgmax)
  #print("labels Flattened",labelsFlattend)   
  return f1_score(labelsFlattend, predictionArgmax, average='macro'),accuracy_score(labelsFlattend, predictionArgmax)



In [0]:
f11,acc=calculateF1Score(yvalid,encoded_Y)

In [146]:
acc

0.7551599587203303

In [147]:
f11

0.6342716238334246

In [0]:
ygiven=[]
ypredicted=[]

In [149]:
!unzip -P ****** -qq '/content/drive/My Drive/EnglishData/test_c_baseline.csv.zip' 

replace test_c_baseline.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

KeyboardInterrupt: ignored

In [0]:
!unzip -P ****** -qq '/content/drive/My Drive/EnglishData/public_data_task_C.zip'

In [0]:
def readData1():
  headers=['id','ypredicted']
  greekdataBaseline = pd.read_csv("test_c_baseline.csv", delimiter=',',names=headers)
  #,converters={"id":convertToInt}       
  #greekdataBaseline.id = greekdataBaseline.id.astype(int)
  #greekdataBaseline=greekdataBaseline[1:]
  print(greekdataBaseline.head())
  print(greekdataBaseline.shape)

  headers=['id','tweet']
  greekDataTest = pd.read_csv("test_c_tweets.tsv", delimiter='\t',names=headers)
                                #converters={"id":convertToInt})
  #greekDataTest=greekDataTest[1:]
  print(greekDataTest.head())
  #print(greekDataTest.dtypes)
  print(greekDataTest.shape)
  result = pd.merge(greekDataTest, greekdataBaseline, on='id', how='inner')
  print(result.head())
  print(result.dtypes)
  print(result.shape)

  #result=
  #result.sort_values(by=['id'], inplace=True)
  #print(result.head())
  dfnumpy=result.to_numpy();
  X=dfnumpy[:, 1].reshape(-1, 1)
  y=dfnumpy[:, 2].reshape(-1, 1)
  tid=dfnumpy[:, 0].reshape(-1, 1)
  preprocessedTweets=X[:,0]
  return preprocessedTweets,y,tid

In [151]:
preprocessedTweets,ypredict1,tid=readData1()

     id ypredicted
0   BC0        IND
1   BC3        IND
2   BC9        IND
3  BC10        IND
4  BC19        IND
(850, 2)
     id                                              tweet
0    id                                              tweet
1   BC0  @USER Lmao bihhh dis what u do to your homies ...
2   BC3  @USER The POTUS is a racist, racist, racist, r...
3   BC9  @USER He then grinned, raising his brow.  Oh d...
4  BC10         Niggas priorities be fucked all the way up
(851, 2)
     id                                              tweet ypredicted
0   BC0  @USER Lmao bihhh dis what u do to your homies ...        IND
1   BC3  @USER The POTUS is a racist, racist, racist, r...        IND
2   BC9  @USER He then grinned, raising his brow.  Oh d...        IND
3  BC10         Niggas priorities be fucked all the way up        IND
4  BC19  You bitches really be walking around talking s...        IND
id            object
tweet         object
ypredicted    object
dtype: object
(850, 3)


In [152]:
type(preprocessedTweets)

numpy.ndarray

In [0]:
preprocessedTweets=preprocessinglib(preprocessedTweets)

In [154]:
Xpredict = tokenizer.texts_to_sequences(preprocessedTweets)
Xpredict = pad_sequences(Xpredict, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', Xpredict.shape)

Shape of data tensor: (850, 250)


In [0]:
ynew = model.predict_classes(Xpredict)

In [0]:
yz=giveLabel2(ypredict1)

In [157]:
set(yz)

{0}

In [0]:
encoded_Ypredict = encoder.transform(yz)


In [0]:
f11,acc=calculateF1Score(ynew,encoded_Ypredict)

In [160]:
f11

0.29312239068336626

In [161]:
acc

0.7847058823529411